# El problema del cubo

> El manual para realizar la práctica, puedes encontrarlo y descargarlo al visitar el siguiente enlace: https://github.com/mariomttz/manuales-ia/blob/master/busquedas_con_restricciones/el_problema_del_cubo.pdf

Los vértices de un cubo se numeran del 1 al 8 de manera que el resultado de sumar los cuatro números de cada cara es el mismo para todas las caras. Se han colocado ya los números 1, 4 y 6 como se muestra en la figura. **¿Qué número va en el vértice marcado con x?**

<div align="center">

![fig_00](./imgs/01.png)

</div>

## Definición de variables

In [ ]:
from itertools import product
from collections import deque

In [ ]:
class Variable:
    def __init__(self, id_, domain):
        self.id = id_
        self.domain = domain
        
    def __str__(self):
        return f'{self.id} -> {self.domain}'

En este caso nuestras variables serán los vertices del cubo, por lo tanto, primero debemos asignar nombre a cada uno de los vertices. Esto se hace para poder definirlos y diferenciarlos durante el algoritmo.

<div align="center">

![fig_01](./imgs/02.png)

</div>

Definimos los valores para cada variable, considerando que algunos vertices ya tienen un valor asignado

In [ ]:
values = [
    ['X1', [1,2,3,4,5,6,7,8]],
    ['X2', [1,2,3,4,5,6,7,8]],
    ['X3', [4]],
    ['X4', [1]],
    ['X5', [1,2,3,4,5,6,7,8]],
    ['X6', [1,2,3,4,5,6,7,8]],
    ['X7', [1,2,3,4,5,6,7,8]],
    ['X8', [6]]
]

In [ ]:
variables = {}

for item in values:
    variables[item[0]] = Variable(*item)

In [ ]:
for v in variables:
    print(variables[v])

## Definición de restricciones

Para este problema tenemos dos restricciones claramente establecidas en las instrucciones

En primer lugar se menciona que la suma de los vertices de las caras debe ser la misma, esto se puede expresar de la siguiente forma:

$$x1 + x2 + x3 + x4 = x5 + x6 + x7 + x8 = x2 + x3 + x6 + x7 = x3 + x4 + x7 + x8 = x1 + x4 + x5 + x8 =  x4 + x3 + x7 + x8$$

Por otro lado, ningún vértice puede repetir asignación de número, por lo tanto:

$$ x1 \neq x2 \neq  x3 \neq  x4 \neq  x5 \neq  x6 \neq  x7 \neq  x8 $$

In [ ]:
def equal_sum(x1, x2, x3, x4, x5, x6, x7, x8):
    sums = [
        x1 + x2 + x3 + x4,
        x5 + x6 + x7 + x8,
        x2 + x3 + x6 + x7,
        x3 + x4 + x7 + x8,
        x1 + x4 + x5 + x8,
        x4 + x3 + x7 + x8
    ]

    return len(set(sums)) == 1

def all_different(x1, x2, x3, x4, x5, x6, x7, x8):
    return len(set([x1, x2, x3, x4, x5, x6, x7, x8])) == 8

In [ ]:
class Constraint:
    
    def __init__(self, id_, rel_variables, func):
        self.id = id_
        self.related_variables = rel_variables
        self.function = func

In [ ]:
constraints_values = [ ['R01', ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8'], equal_sum],
                       ['R02', ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8'], all_different]]

In [ ]:
constraints = {}

for item in constraints_values:
    constraints[item[0]] = Constraint(*item)

## Reducción de dominio

Creamos todas las aristas *(X, C)* para todas las variables y restricciones

In [ ]:
edges = []

for key in constraints:
    for var in constraints[key].related_variables:
        if (var, key) not in edges:
            edges.append((var, key))

Realizamos el algoritmo AC-3 para la consistencia de arco

In [ ]:
edges_deque = deque(edges)

while(edges_deque):
    variable_name, constraint_name = edges_deque.popleft()

    for value in list(variables[variable_name].domain):
        valid = False

        for combination in product(*[variables[v].domain if v != variable_name else [value] for v in constraints[constraint_name].related_variables]):
            if constraints[constraint_name].function(*combination):
                valid = True
                break

        if not valid:
            variables[variable_name].domain.remove(value)
            related_cons = [constraint for variable, constraint in edges if variable == variable_name]
            affected_edges = [(v, c) for v, c in edges if c in related_cons and v != variable_name]
            edges_deque += deque([edge for edge in affected_edges if edge not in edges_deque])


Imprimimos los valores restantes después de la reducción de dominio

In [ ]:
for v in variables:
    print(variables[v])

## Búsqueda

En este caso como cada variable sólo tiene un valor restante, sólo tenemos una combinación por probar. Si esta combinación cumple con las restricciones, entonces la solución es la combinación. En caso contrario, el problema no tendría solución.

In [ ]:
equal_sum(8,5,4,1,3,2,7,6) and all_different(8,5,4,1,3,2,7,6)

Por lo tanto la solución es esa única asignación posible

In [ ]:
for v in variables:
    print(f'{v}: {variables[v].domain[0]}')

Y respondiendo a la pregunta del problema, el vertice marcado con una $x$ (que corresponde a nuestra variable $x_{6}$) debe tomar el valor **2** para cumplir con las restricciones del problema

**Ejercicio 1:**

Ahora consideremos números en las aristas en vez de los vertices, si tomamos numeros del 1 al 12 ¿Podemos asignar un número diferente a cada arista de manera que la suma de las aristas de todas las caras sumen lo mismo considerando que _H = 2_ , _G = 11_ y _F = 12_?
Encuentra todas las soluciones posibles si es que hay

<div align="center">

![fig_02](./imgs/03.png)

</div>